# TM10007 Assignment template

In [6]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [1]:
# Data loading functions. Uncomment the one you want to use
#from worcgist.load_data import load_data
from worclipo.load_data import load_data
#from worcliver.load_data import load_data
#from ecg.load_data import load_data

# Import classifiers
from sklearn import model_selection

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

X = data.drop("label",axis=1)
y = data["label"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)


The number of samples: 115
The number of columns: 494


### Preprocessing

Missing data
* Part 1: Finding missing data

In [2]:
# Define missing value indicators
custom_missing = ['NA', 'null', 'N/A', '?', 'None', 'none', '-']

# Count NaNs
nan_counts = X_train.isna().sum()
nan_counts_selected = nan_counts[nan_counts !=0]

# Count empty strings
empty_string_counts = (X_train == '').sum()
empty_string_counts_selected = empty_string_counts[empty_string_counts != 0]

# Count zero's
zero_counts = (X_train == 0).sum()
zero_counts_selected = zero_counts[zero_counts != 0]

# Count custom missing indicators (case-insensitive match)
custom_missing_counts = X_train.apply(lambda col: col.astype(str).str.lower().isin([val.lower() for val in custom_missing]).sum())
custom_missing_counts_selected = custom_missing_counts[custom_missing_counts !=0]

# Printing
print(nan_counts_selected)
print(empty_string_counts_selected)
print(zero_counts_selected)
print(custom_missing_counts_selected)
#X_train_zero_columns = X_train[zero_counts_selected.index]
#X_train_zero_columns

Series([], dtype: int64)
Series([], dtype: int64)
PREDICT_original_sf_area_min_2.5D                                       1
PREDICT_original_tf_LBP_min_R3_P12                                     86
PREDICT_original_tf_LBP_min_R8_P24                                     73
PREDICT_original_tf_LBP_quartile_range_R8_P24                           5
PREDICT_original_tf_LBP_min_R15_P36                                    59
PREDICT_original_tf_LBP_peak_R15_P36                                    1
PREDICT_original_tf_LBP_peak_position_R15_P36                           1
PREDICT_original_tf_LBP_quartile_range_R15_P36                         45
PREDICT_original_vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0                84
PREDICT_original_vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0             82
PREDICT_original_vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0               86
PREDICT_original_vf_Frangi_full_peak_position_SR(1.0, 10.0)_SS2.0      86
PREDICT_original_vf_Frangi_full_quartile_range_SR(1.0, 10.0)_S

* Part 2: Processing missing data

In [3]:
# Missing data was only find in the form of zero's, therefore only zero_counts is further used.
limit = len(X_train.index)*50/100 #If 50% or more of the data within one feature is missing the feature is deleted
valid_columns = [col for col, count in zero_counts.items() if count < limit]

# Keep only the valid columns in both X_train and X_test
X_train = X_train[valid_columns]
X_test = X_test[valid_columns]

# Imputate remaining zero's
# Import
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import shapiro

# Replace zeros with NaN
X_train.replace(0, np.nan, inplace=True)
X_test.replace(0, np.nan, inplace=True)

imputation_strategies = {}  # Store mean/median decision per column

for col in X_train.select_dtypes(include=['number']).columns:  # Only numeric columns
    col_data = X_train[col].dropna()  # Remove NaN values for testing

    if len(col_data) > 3:  # Shapiro requires at least 3 non-null values
        if col_data.nunique() == 1:  # Check if all values are the same
            strategy = 'median'  # Default to median if no variability
        else:
            _, p = shapiro(col_data)
            strategy = 'mean' if p > 0.05 else 'median'
    else:
        strategy = 'median'  # Default to median if too few values

    imputation_strategies[col] = strategy

# Create imputers for mean and median
mean_imputer = SimpleImputer(strategy='mean')
median_imputer = SimpleImputer(strategy='median')

for col, strategy in imputation_strategies.items():
    imputer = mean_imputer if strategy == 'mean' else median_imputer
    X_train[[col]] = imputer.fit_transform(X_train[[col]])
    X_test[[col]] = imputer.transform(X_test[[col]])  # Use the same imputer

Scaling data

In [4]:
import pandas as pd
from sklearn import preprocessing

scaler_MaxAbsScaler = preprocessing.MaxAbsScaler()
scaler_Normalizer = preprocessing.Normalizer()
scaler_standard = preprocessing.StandardScaler()
scaler_minmax = preprocessing.MinMaxScaler()
scaler_robust = preprocessing.RobustScaler()

scaled_MaxAbsScaler_array_train = scaler_MaxAbsScaler.fit_transform(X_train)
scaled_MaxAbsScaler_array_test = scaler_MaxAbsScaler.transform(X_test)
scaled_Normalizer_array_train = scaler_Normalizer.fit_transform(X_train)
scaled_Normalizer_array_test = scaler_Normalizer.transform(X_test)
scaled_standard_array_train = scaler_standard.fit_transform(X_train)
scaled_standard_array_test = scaler_standard.transform(X_test)
scaled_minmax_array_train = scaler_minmax.fit_transform(X_train)
scaled_minmax_array_test = scaler_minmax.transform(X_test)
scaled_robust_array_train = scaler_robust.fit_transform(X_train)
scaled_robust_array_test = scaler_robust.transform(X_test)

X_scaled_MaxAbsScaler_train = pd.DataFrame(scaled_MaxAbsScaler_array_train, columns=X_train.columns)
X_df_scaled_Normalizer_train = pd.DataFrame(scaled_Normalizer_array_train, columns=X_train.columns)
X_scaled_Normalizer_test = pd.DataFrame(scaled_Normalizer_array_test, columns=X_test.columns)
X_scaled_standard_train = pd.DataFrame(scaled_standard_array_train, columns=X_train.columns)
X_scaled_standard_test = pd.DataFrame(scaled_standard_array_test, columns=X_test.columns)
X_scaled_minmax_train = pd.DataFrame(scaled_minmax_array_train, columns=X_train.columns)
X_scaled_minmax_test = pd.DataFrame(scaled_minmax_array_test, columns=X_test.columns)
X_scaled_robust_train = pd.DataFrame(scaled_robust_array_train, columns=X_train.columns)
X_scaled_robust_test = pd.DataFrame(scaled_robust_array_test, columns=X_test.columns)


Feature extraction

Lloyd

Inge

Random forest

In [5]:
def colorplot(clf, ax, x, y, h=100):
    '''
    Overlay the decision areas as colors in an axes.

    Input:
        clf: trained classifier
        ax: axis to overlay color mesh on
        x: feature on x-axis
        y: feature on y-axis
        h(optional): steps in the mesh
    '''
    # Create a meshgrid the size of the axis
    xstep = (x.max() - x.min() ) / 20.0
    ystep = (y.max() - y.min() ) / 20.0
    x_min, x_max = x.min() - xstep, x.max() + xstep
    y_min, y_max = y.min() - ystep, y.max() + ystep
    h = max((x_max - x_min, y_max - y_min))/h
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    if hasattr(clf, "decision_function"):
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    if len(Z.shape) > 1:
        Z = Z[:, 1]

    # Put the result into a color plot
    cm = plt.cm.RdBu_r
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)
    del xx, yy, x_min, x_max, y_min, y_max, Z, cm

In [18]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

# Define classifiers
homemade_random_forest = BaggingClassifier(DecisionTreeClassifier())

clsfs = [
    ("Decision Tree", DecisionTreeClassifier()),
    ("Random Forest", RandomForestClassifier()),
    ("Homemade Random Forest", homemade_random_forest)
]

# Encode categorical labels if needed
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  # Convert to numeric labels

# Evaluate classifiers with cross-validation
for name, clf in clsfs:
    # Perform 5-fold cross-validation
    scores = cross_val_score(clf, X_train, y_train_encoded, cv=5)
    
    # Compute mean accuracy
    mean_acc = np.mean(scores)
    
    # Print the accuracy
    print(f"{name} - Mean Accuracy (5-fold CV): {mean_acc:.2f}")

Decision Tree - Mean Accuracy (5-fold CV): 0.72
Random Forest - Mean Accuracy (5-fold CV): 0.73
Homemade Random Forest - Mean Accuracy (5-fold CV): 0.76
